In [12]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))

df=pd.DataFrame(df[0])

In [13]:
df

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
283,M8Z,Etobicoke,Mimico NW
284,M8Z,Etobicoke,The Queensway West
285,M8Z,Etobicoke,Royal York South West
286,M8Z,Etobicoke,South of Bloor


In [14]:
df=df[df.Borough!='Not assigned']
df.reset_index(inplace=True)

In [15]:
for i in range(df.shape[0]):
    if df.loc[i,'Neighbourhood']=='Not assigned':
        df.loc[i,'Neighbourhood']=df.loc[i,'Borough']

//anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [16]:
df = df.groupby('Borough').agg({'Postcode':'first',
                             'Neighbourhood': ', '.join 
                             }).reset_index()
df

,Borough,Postcode,Neighbourhood
0,Central Toronto,M4N,"Lawrence Park, Roselawn, Davisville North, For..."
1,Downtown Toronto,M5A,"Harbourfront, Regent Park, Ryerson, Garden Dis..."
2,East Toronto,M4E,"The Beaches, The Danforth West, Riverdale, The..."
3,East York,M4B,"Woodbine Gardens, Parkview Hill, Woodbine Heig..."
4,Etobicoke,M9A,"Islington Avenue, Cloverdale, Islington, Marti..."
5,Mississauga,M7R,Canada Post Gateway Processing Centre
6,North York,M3A,"Parkwoods, Victoria Village, Lawrence Heights,..."
7,Queen's Park,M7A,Queen's Park
8,Scarborough,M1B,"Rouge, Malvern, Highland Creek, Rouge Hill, Po..."
9,West Toronto,M6H,"Dovercourt Village, Dufferin, Little Portugal,..."


In [17]:
df.shape


(11, 3)

In [18]:
df1=pd.read_csv('https://cocl.us/Geospatial_data')
df1

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [20]:
df1=df1.rename(columns={'Postal Code':'Postcode'})
df2=pd.merge(df,df1,on='Postcode')

In [21]:
df2

,Borough,Postcode,Neighbourhood,Latitude,Longitude
0,Central Toronto,M4N,"Lawrence Park, Roselawn, Davisville North, For...",43.728020,-79.388790
1,Downtown Toronto,M5A,"Harbourfront, Regent Park, Ryerson, Garden Dis...",43.654260,-79.360636
2,East Toronto,M4E,"The Beaches, The Danforth West, Riverdale, The...",43.676357,-79.293031
3,East York,M4B,"Woodbine Gardens, Parkview Hill, Woodbine Heig...",43.706397,-79.309937
4,Etobicoke,M9A,"Islington Avenue, Cloverdale, Islington, Marti...",43.667856,-79.532242
5,Mississauga,M7R,Canada Post Gateway Processing Centre,43.636966,-79.615819
6,North York,M3A,"Parkwoods, Victoria Village, Lawrence Heights,...",43.753259,-79.329656
7,Queen's Park,M7A,Queen's Park,43.662301,-79.389494
8,Scarborough,M1B,"Rouge, Malvern, Highland Creek, Rouge Hill, Po...",43.806686,-79.194353
9,West Toronto,M6H,"Dovercourt Village, Dufferin, Little Portugal,...",43.669005,-79.442259


Below I would add some mapping information for the segment and clustering to show how  
those areas from 'Borough' are different from each other.

In [26]:
from geopy.geocoders import Nominatim
address = 'Toronto,CA'

geolocator = Nominatim(user_agent="TR_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
#print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

After setting the location, lat, and long, lets see the mapping and the segment results. 

In [30]:
import folium
map_tor = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df2['Latitude'], df2['Longitude'], df2['Borough'], df2['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tor)  
    
map_tor